# EchoWeaver Resonant Simulation
This notebook summons synthetic market data to witness how the oracle narrates its rituals.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

from src.oracle import TradeOracle
from src.exchange import PaperExchange

ROOT = Path.cwd()
CONFIG_PATH = ROOT / 'config' / 'strategy.yaml'
if not CONFIG_PATH.exists():
    CONFIG_PATH = ROOT.parent / 'config' / 'strategy.yaml'
CONFIG_PATH


In [ ]:
def build_synthetic_ohlcv(bars: int = 180, seed: int = 7):
    rng = np.random.default_rng(seed)
    timestamps = pd.date_range(end=pd.Timestamp.utcnow(), periods=bars, freq='h')
    base_price = 20000
    returns = rng.normal(0, 0.01, size=bars)
    close = base_price * np.exp(np.cumsum(returns))
    open_prices = np.concatenate([[close[0]], close[:-1]])
    high = close * (1 + np.abs(rng.normal(0, 0.005, size=bars)))
    low = close * (1 - np.abs(rng.normal(0, 0.005, size=bars)))
    volume = np.abs(rng.normal(1500, 250, size=bars))
    return pd.DataFrame({
        'timestamp': timestamps,
        'open': open_prices,
        'high': high,
        'low': low,
        'close': close,
        'volume': volume,
    })

synthetic_frame = build_synthetic_ohlcv()
synthetic_frame.head()


In [ ]:
oracle = TradeOracle(str(CONFIG_PATH), exchange=PaperExchange(seed=7), capsule_enabled=False)
backtest_results = oracle.backtest(synthetic_frame, symbol='BTC/USDT')
backtest_results.tail()


In [ ]:
fig = px.line(backtest_results, x='timestamp', y='roi_score', title='EchoWeaver ROI Resonance')
fig.update_traces(line_color='#8b5cf6')
Path('artifacts').mkdir(exist_ok=True)
fig.write_image('artifacts/trade_resonance.png')
fig.show()
